In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [5]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from scipy.stats import boxcox
from matplotlib import pyplot

In [3]:
df_path = "/home/rupeshr/Desktop/TSA_Python/dataset/multivariate_dataset.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- Date: string (nullable = true)
 |-- Money: double (nullable = true)
 |-- Spending: double (nullable = true)
 |-- Gain: integer (nullable = true)
 |-- Loss: integer (nullable = true)

+----------+------+--------+----+----+
|      Date| Money|Spending|Gain|Loss|
+----------+------+--------+----+----+
|1995-01-01|3492.4|  4851.2|2891|1000|
|1995-02-01|3489.9|  4850.8|2893|1101|
|1995-03-01|3491.1|  4885.4|2895|1202|
|1995-04-01|3499.2|  4890.2|2897|1303|
|1995-05-01|3524.2|  4933.1|   0|1404|
|1995-06-01|3548.9|  4977.5|   0|1505|
|1995-07-01|3567.4|  4970.2|   0|1606|
|1995-08-01|3589.0|  5005.3|   0|1707|
|1995-09-01|3602.1|  5020.5|   0|1808|
|1995-10-01|3613.4|  5013.9|   0|1909|
|1995-11-01|3619.9|  5055.6|   0|2010|
|1995-12-01|3629.5|  5097.5|   0|2111|
|1996-01-01|3647.9|  5085.7|   0|2212|
|1996-02-01|3661.9|  5132.8|2917|2313|
|1996-03-01|3687.0|  5173.3|2919|2414|
|1996-04-01|3697.8|  5208.0|2921|   0|
|1996-05-01|3709.7|  5223.8|2923|   0|
|1996-06-01|3722.7|  5229.8

In [14]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import Window
from rest_framework.response import Response
# Auxiliary functions
# Pandas Types -> Spark Types
#from edge.stats.sparksession import spark_session

def equivalent_type(f):
    if f == 'datetime64[ns]': return DateType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    else: return StringType()
def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)
# Converts pandas dataframe to spark dataframe
def pandas_to_spark(df_pandas):
    sqlContext = spark
    columns = list(df_pandas.columns)
    types = list(df_pandas.dtypes)
    struct_list = []
    for column, typo in zip(columns, types):
        struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(df_pandas, p_schema)






def box_and_cox(*,df,column,lambdavalue):
    for i in df.columns:
        if i in column:
            s = df.select(i).dtypes[0]
            try:
                if s[1] == "int" or s[1] == "float" or s[1] == "double" or s[1] == "bigint" or s[1] == "long":
                    temp_df=df.select(i).toPandas()
                    temp_df[column]=boxcox(temp_df[column],lmbda=lambdavalue)
                    data=pandas_to_spark(temp_df)
            except:
                return Response({"status": "failure", "error": "please select numerical type column"})
    return data



#Below are some common values for lambda

#lambda = -1. is a reciprocal transform.
#lambda = -0.5 is a reciprocal square root transform.
#lambda = 0.0 is a log transform.
#lambda = 0.5 is a square root transform.
#lambda = 1.0 is no transform.

In [24]:
df_path = "/home/rupeshr/Desktop/TSA_Python/dataset/time series dataset/archive/monthly-beer-production-in-austr.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+-------+-----------------------+
|  Month|Monthly beer production|
+-------+-----------------------+
|1956-01|                   93.2|
|1956-02|                   96.0|
|1956-03|                   95.2|
|1956-04|                   77.1|
|1956-05|                   70.9|
|1956-06|                   64.8|
|1956-07|                   70.1|
|1956-08|                   77.3|
|1956-09|                   79.5|
|1956-10|                  100.6|
|1956-11|                  100.7|
|1956-12|                  107.1|
|1957-01|                   95.9|
|1957-02|                   82.8|
|1957-03|                   83.3|
|1957-04|                   80.0|
|1957-05|                   80.4|
|1957-06|                   67.5|
|1957-07|                   75.7|
|1957-08|                   71.1|
+-------+-----------------------+
only showing top 20 rows



In [30]:
data=box_and_cox(df=df,column='Monthly beer production',lambdavalue=0.0)

In [31]:
df.show()

+-------+-----------------------+
|  Month|Monthly beer production|
+-------+-----------------------+
|1956-01|                   93.2|
|1956-02|                   96.0|
|1956-03|                   95.2|
|1956-04|                   77.1|
|1956-05|                   70.9|
|1956-06|                   64.8|
|1956-07|                   70.1|
|1956-08|                   77.3|
|1956-09|                   79.5|
|1956-10|                  100.6|
|1956-11|                  100.7|
|1956-12|                  107.1|
|1957-01|                   95.9|
|1957-02|                   82.8|
|1957-03|                   83.3|
|1957-04|                   80.0|
|1957-05|                   80.4|
|1957-06|                   67.5|
|1957-07|                   75.7|
|1957-08|                   71.1|
+-------+-----------------------+
only showing top 20 rows



In [32]:
var=['column','spen']
print(len(var))

2
